<a href="https://www.kaggle.com/code/aisuko/ft-gpt-2-detector-for-binary-classification?scriptVersionId=210734467" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -U -q evaluate

In [2]:
import os

os.environ['DATASET']='aisuko/phishing-binary-classification'
# os.environ['ORIGINAL_MODEL']="nvidia/Hymba-1.5B-Base"
os.environ['ORIGINAL_MODEL']="openai-community/roberta-large-openai-detector"


In [3]:
from datasets import load_dataset

ds = load_dataset(os.getenv('DATASET'))
ds

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/528006 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/66001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/66001 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'labels'],
        num_rows: 528006
    })
    validation: Dataset({
        features: ['url', 'labels'],
        num_rows: 66001
    })
    test: Dataset({
        features: ['url', 'labels'],
        num_rows: 66001
    })
})

In [4]:
from datasets import DatasetDict

pre_processed_ds_train_low=ds['train'].shuffle(seed=42).select(range(10000))
pre_processed_ds_test_low=ds['test'].shuffle(seed=42).select(range(5000))
pre_processed_ds_validate_low=ds['validation'].shuffle(seed=42).select(range(5000))

ds_low=DatasetDict({
    'train': pre_processed_ds_train_low,
    'test': pre_processed_ds_test_low,
    'validation': pre_processed_ds_validate_low,
})

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer=AutoTokenizer.from_pretrained(os.getenv('ORIGINAL_MODEL'))
tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


RobertaTokenizerFast(name_or_path='openai-community/roberta-large-openai-detector', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

# Pre-processing data

In [6]:
# define text preprocessing
def preprocess_func(examples):
    return tokenizer(examples['url'], truncation=True)

# apply to every row of data
tokenized_dataset=ds.map(preprocess_func, batched=True)
tokenized_dataset

Map:   0%|          | 0/528006 [00:00<?, ? examples/s]

Map:   0%|          | 0/66001 [00:00<?, ? examples/s]

Map:   0%|          | 0/66001 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 528006
    })
    validation: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 66001
    })
    test: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 66001
    })
})

In [7]:
tokenized_ds_low=ds_low.map(preprocess_func, batched=True)
tokenized_ds_low

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['url', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [8]:
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}


model=AutoModelForSequenceClassification.from_pretrained(
    os.getenv('ORIGINAL_MODEL'),
    num_labels=2,
    id2label=id2label, 
    label2id=label2id
)
model.to("cuda")

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at openai-community/roberta-large-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [9]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight True
roberta.embeddings.position_embeddings.weight True
roberta.embeddings.token_type_embeddings.weight True
roberta.embeddings.LayerNorm.weight True
roberta.embeddings.LayerNorm.bias True
roberta.encoder.layer.0.attention.self.query.weight True
roberta.encoder.layer.0.attention.self.query.bias True
roberta.encoder.layer.0.attention.self.key.weight True
roberta.encoder.layer.0.attention.self.key.bias True
roberta.encoder.layer.0.attention.self.value.weight True
roberta.encoder.layer.0.attention.self.value.bias True
roberta.encoder.layer.0.attention.output.dense.weight True
roberta.encoder.layer.0.attention.output.dense.bias True
roberta.encoder.layer.0.attention.output.LayerNorm.weight True
roberta.encoder.layer.0.attention.output.LayerNorm.bias True
roberta.encoder.layer.0.intermediate.dense.weight True
roberta.encoder.layer.0.intermediate.dense.bias True
roberta.encoder.layer.0.output.dense.weight True
roberta.encoder.layer.0.output.dense.bia

# Freeze the pre-trained weights

In [10]:
for name, param in model.base_model.named_parameters():
    param.requires_grad=False

In [11]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight False
roberta.embeddings.position_embeddings.weight False
roberta.embeddings.token_type_embeddings.weight False
roberta.embeddings.LayerNorm.weight False
roberta.embeddings.LayerNorm.bias False
roberta.encoder.layer.0.attention.self.query.weight False
roberta.encoder.layer.0.attention.self.query.bias False
roberta.encoder.layer.0.attention.self.key.weight False
roberta.encoder.layer.0.attention.self.key.bias False
roberta.encoder.layer.0.attention.self.value.weight False
roberta.encoder.layer.0.attention.self.value.bias False
roberta.encoder.layer.0.attention.output.dense.weight False
roberta.encoder.layer.0.attention.output.dense.bias False
roberta.encoder.layer.0.attention.output.LayerNorm.weight False
roberta.encoder.layer.0.attention.output.LayerNorm.bias False
roberta.encoder.layer.0.intermediate.dense.weight False
roberta.encoder.layer.0.intermediate.dense.bias False
roberta.encoder.layer.0.output.dense.weight False
roberta.encoder.layer.

In [12]:
from transformers import DataCollatorWithPadding

data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation

In [13]:
# Original code from https://github.com/ShawhinT/YouTube-Blog/blob/main/LLMs/model-compression/0_train-teacher.ipynb
# Acknowledge the original author
# Adapte by Bowen

import evaluate
import numpy as np


accuracy=evaluate.load("accuracy")
auc_score=evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    predictions, labels=eval_pred
    probabilities=np.exp(predictions)/ np.exp(predictions).sum(-1, keepdims=True)
    positive_class_probs=probabilities[:,1]
    # compute auc
    auc=np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

# Train model

In [14]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="phishing-binary-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="tensorboard",
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset["train"],    
    # eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_ds_low["train"],
    eval_dataset=tokenized_ds_low["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.550100,0.401487,0.818000,0.927000
2,0.461100,0.360547,0.842000,0.923000
3,0.444500,0.375922,0.827000,0.939000
4,0.413000,0.305807,0.866000,0.946000
5,0.415200,0.355411,0.837000,0.953000
6,0.408600,0.290823,0.874000,0.949000
7,0.405700,0.333821,0.853000,0.946000
8,0.396600,0.280712,0.880000,0.953000
9,0.396100,0.283576,0.878000,0.952000
10,0.396200,0.281332,0.882000,0.954000


TrainOutput(global_step=12500, training_loss=0.428705703125, metrics={'train_runtime': 1190.3421, 'train_samples_per_second': 84.009, 'train_steps_per_second': 10.501, 'total_flos': 1.0533197918297376e+16, 'train_loss': 0.428705703125, 'epoch': 10.0})

# Apply Model to Validation Dataset

In [15]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_ds_low["validation"])

In [16]:
# extract the logits and labels from the predictions object
logits=predictions.predictions
labels=predictions.label_ids

metrics=compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.893, 'AUC': 0.96}


# Push to the HuggingFace

In [17]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
kwargs={
    'model_name': os.getenv('DATASET'),
    'finetuned_from': os.getenv('ORIGINAL_MODEL'),
    'tasks': 'Text-Generation',
    'dataset': os.getenv('DATASET')
}

tokenizer.push_to_hub(os.getenv('DATASET'))
trainer.push_to_hub('**kwargs')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1733115499.e71f15d8e01b.23.0:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aisuko/phishing-binary-classification/commit/82d268e323ac7de88286bcf1c30752241cc33385', commit_message='**kwargs', commit_description='', oid='82d268e323ac7de88286bcf1c30752241cc33385', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aisuko/phishing-binary-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='aisuko/phishing-binary-classification'), pr_revision=None, pr_num=None)

# References

* https://www.kaggle.com/code/aisuko/fine-tune-openelm-270m-with-orpo


# Acknowledge

* https://github.com/ShawhinT/YouTube-Blog/blob/main/LLMs/model-compression/0_train-teacher.ipynb